In [1]:
from keras import applications
from keras import Sequential
from keras.layers.core import Dense, Activation


Using TensorFlow backend.


In [2]:
model = applications.VGG16(weights='imagenet', include_top=True)

In [3]:
model.layers[-1].get_config()

{'activation': 'softmax',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'predictions',
 'trainable': True,
 'units': 1000,
 'use_bias': True}

In [4]:
model.layers.pop()

In [5]:
model.layers

In [6]:
model.layers[-1].get_config()

{'activation': 'relu',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'fc2',
 'trainable': True,
 'units': 4096,
 'use_bias': True}

In [7]:
top_model = Sequential()
top_model.add(Dense(10,input_shape=(4096,)))
top_model.add(Activation('softmax'))

In [8]:
# CREATE AN "REAL" MODEL FROM VGG16
# BY COPYING ALL THE LAYERS OF VGG16
new_model = Sequential()
for l in model.layers:
    new_model.add(l)

In [9]:
# CONCATENATE THE TWO MODELS
new_model.add(top_model)

In [10]:
for layer in new_model.layers[:-1]:
    layer.trainable = False

In [11]:
print (new_model.layers[-2].get_config())
print()
print (new_model.layers[-1].get_config())

{'kernel_constraint': None, 'activation': 'relu', 'name': 'fc2', 'bias_constraint': None, 'kernel_regularizer': None, 'activity_regularizer': None, 'bias_regularizer': None, 'kernel_initializer': {'config': {'scale': 1.0, 'seed': None, 'distribution': 'uniform', 'mode': 'fan_avg'}, 'class_name': 'VarianceScaling'}, 'use_bias': True, 'trainable': False, 'bias_initializer': {'config': {}, 'class_name': 'Zeros'}, 'units': 4096}

[{'config': {'kernel_constraint': None, 'activation': 'linear', 'name': 'dense_1', 'bias_constraint': None, 'kernel_regularizer': None, 'activity_regularizer': None, 'bias_regularizer': None, 'kernel_initializer': {'config': {'scale': 1.0, 'distribution': 'uniform', 'mode': 'fan_avg', 'seed': None}, 'class_name': 'VarianceScaling'}, 'use_bias': True, 'dtype': 'float32', 'batch_input_shape': (None, 4096), 'trainable': True, 'bias_initializer': {'config': {}, 'class_name': 'Zeros'}, 'units': 10}, 'class_name': 'Dense'}, {'config': {'trainable': True, 'activation': '

In [12]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()


In [13]:
from keras.utils import np_utils

img_rows, img_cols = 32, 32
nb_classes = 10

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols,3)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols,3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train,nb_classes)
Y_test = np_utils.to_categorical(y_test,nb_classes)

In [14]:
X_train.shape

(50000, 32, 32, 3)

In [15]:
from scipy.misc import imresize
import numpy as np
X_train_resized = np.zeros(shape = (1000,224,224,3))
X_test_resized = np.zeros(shape = (100,224,224,3))

for im_ind in range(1000):
    X_train_resized[im_ind] = imresize(X_train[im_ind],(224,224))
for im_ind in range(100):   
    X_test_resized[im_ind] = imresize(X_test[im_ind],(224,224))

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/ipykernel/__main__.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [16]:
new_model.compile(loss='categorical_crossentropy',
          optimizer='sgd',
          metrics=['accuracy'])

new_model.fit(X_train_resized, Y_train[:1000], batch_size=32, 
          nb_epoch=2,verbose=1,
          validation_data=(X_test_resized, Y_test[:100]))

/home/ubuntu/anaconda2/envs/deep-learning/lib/python3.5/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 1000 samples, validate on 100 samples
Epoch 1/2
 480/1000 [=============>................] - ETA: 12:48 - loss: 4.9569 - acc: 0.2292

KeyboardInterrupt: 